In [4]:
print(packages_import_statement)
import requests


import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

import re
import time 

from pprint import pprint




In [5]:
#https://data.cityofnewyork.us/City-Government/DSNY-Monthly-Tonnage-Data/ebb7-mvp5
url =  "https://data.cityofnewyork.us/resource/ebb7-mvp5.json"
params = {"$limit":25000}
r = requests.get(url = url,params=params)
data = r.json()

#### #### #### #### #### #### #### #### 
nyc_data_id = "removed"
nyc_secret_key = "removed"

app_token = "removed"
secret_token="removed"
#### #### #### #### #### #### #### #### 

In [6]:
df = pd.DataFrame(data)
print(df.dtypes)
df.describe()

month                  object
borough                object
communitydistrict      object
refusetonscollected    object
papertonscollected     object
mgptonscollected       object
borough_id             object
resorganicstons        object
schoolorganictons      object
leavesorganictons      object
xmastreetons           object
dtype: object


,month,borough,communitydistrict,refusetonscollected,papertonscollected,mgptonscollected,borough_id,resorganicstons,schoolorganictons,leavesorganictons,xmastreetons
count,21166,21166,21166,21166,18544,18256,21166,1613,1030,737,1480
unique,367,5,18,20521,16638,15720,5,1534,948,724,822
top,2012 / 03,Brooklyn,03,3057.000000000000000,180.789999999999992,238.530000000000001,3,12.370000000000001,29.100000000000001,27.850000000000001,16.800000000000001
freq,59,6399,1797,4,4,5,6399,4,6,2,18


In [7]:
df['boroughid_communitydistrict'] = df['borough_id']+"__"+df['communitydistrict']
df['boroughid_communitydistrict'].value_counts().head(5)

5__02    367
5__01    366
5__03    366
1__03    365
1__10    364
Name: boroughid_communitydistrict, dtype: int64

In [8]:
df['papertonscollected']=df['papertonscollected'].astype(float)
df['mgptonscollected']=df['mgptonscollected'].astype(float)
df['refusetonscollected']=df['refusetonscollected'].astype(float)

In [10]:
df['month_clean'] = df['month'].apply(lambda x: x.split("/")[1]).astype(int)
df['year_clean'] = df['month'].apply(lambda x: x.split("/")[0]).astype(int)
df[['month','month_clean','year_clean']].sample(2)

,month,month_clean,year_clean
14982,1999 / 10,10,1999
12949,2002 / 08,8,2002


In [11]:
#introduce a custom function performing distribution analysis
def distribution_analysis(x, log_scale = False, fit_distribution = 'None', bins = 50, vis_means = True, vis_curve = True, print_outputs = True):
    ''' x - array of observations
    log_scale - analyze distribution of log(x) if True
    fit_distribution - fit the distribution ('normal', 'gev' or 'pareto') or do nothing if 'None'
    bins - how many bins to use for binning the data
    vis_means - show mean and std lines if True
    vis_curve - show interpolated distribution curve over the histogram bars if True
    print_outputs - print mean, std and percentiles
    '''
    
    if log_scale: 
        x1 = np.log10(x) #convert data to decimal logarithms
        xlabel = 'log(values)' #reflect in x labels
    else:
        x1 = x #leave original scale 
        xlabel = 'values'
    mu = x1.mean() #compute the mean
    if log_scale: #if logscale, output all three - log mean, its original scale and original scale mean
        print('Log mean = {:.2f}({:.2f}), mean = {:.2f}'.format(mu,10**mu,x.mean()))
    else:
        print('Mean = {:.2f}'.format(mu)) #otherwise print mean
    sigma = x1.std() #compute and output standard deviation 
    print('Standard deviation = {:.2f}'.format(sigma))
    for p in [1,5,25,50,75,95,99]: #output percentile values
        print('{:d} percentile = {:.2f}'.format(p,np.percentile(x,p)))
        
    #visualize histogram and the interpolated line (if vis_curve=True) using seaborn
    sns.distplot(x1, hist=True, kde=vis_curve, 
        bins=bins,color = 'darkblue', 
        hist_kws={'edgecolor':'black'},
        kde_kws={'linewidth': 4})
    
    #show vertical lines for mean and std if vis_means = True
    if vis_means:
        plt.axvline(mu, color='r', ls='--', lw=2.0)
        plt.axvline(mu-sigma, color='g', ls='--', lw=2.0)
        plt.axvline(mu+sigma, color='g', ls='--', lw=2.0)
        
    ylim = plt.gca().get_ylim() #keep the y-range of original distribution density values 
    #(to make sure the fitted distribution would not affect it)
    
    h = np.arange(mu - 3 * sigma, mu + 3 * sigma, sigma / 100) #3-sigma visualization range for the fitted distribution
    pars = None #fitted distribution parameters
    
    #fit and visualize the theoretic distribution
    if fit_distribution == 'normal':
        pars = norm.fit(x1)
        plt.plot(h,norm.pdf(h,*pars),'r')
    elif fit_distribution == 'gev':
        pars = gev.fit(x1)
        plt.plot(h,gev.pdf(h,*pars),'r')
    elif fit_distribution == 'pareto':
        pars = pareto.fit(x1)
        plt.plot(h,pareto.pdf(h,*pars),'r')
    
    plt.xlabel(xlabel) #add x label 
    plt.ylim(ylim) #restore the y-range of original distribution density values 
    plt.show()
    return pars

In [20]:
#Pre-Lockdown Diversion Rates

#Jan 2018 - Feb 2020

pre_lockdown_stats=\
df[(df['year_clean'].isin([2018,2019]))|
  ((df['year_clean'].isin([2020]))&(df['month_clean'].isin([1,2])))]\
[['boroughid_communitydistrict','month_clean','year_clean','papertonscollected','mgptonscollected','refusetonscollected']]\
.groupby(by=['boroughid_communitydistrict']).agg({"papertonscollected":"sum"
                                                  ,"mgptonscollected":"sum"
                                                  ,"refusetonscollected":"sum"})
pre_lockdown_stats['diversion_rate'] = (pre_lockdown_stats['papertonscollected']\
                        +pre_lockdown_stats['mgptonscollected'])/pre_lockdown_stats['refusetonscollected']

#Post-Lockdown Diversion Rates

#March 2020 - Nov 2020

post_lockdown_stats=\
df[(df['year_clean'].isin([2020]))&(df['month_clean']>=3)]\
[['boroughid_communitydistrict','month_clean','year_clean','papertonscollected','mgptonscollected','refusetonscollected']]\
.groupby(by=['boroughid_communitydistrict']).agg({"papertonscollected":"sum"
                                                  ,"mgptonscollected":"sum"
                                                  ,"refusetonscollected":"sum"})
post_lockdown_stats['diversion_rate'] = (post_lockdown_stats['papertonscollected']\
                        +post_lockdown_stats['mgptonscollected'])/post_lockdown_stats['refusetonscollected']


delta = pre_lockdown_stats[['diversion_rate']]\
.merge(right=post_lockdown_stats[['diversion_rate']],left_index=True,right_index=True,suffixes=("_pre","_post"))
delta['Change_Diversion_Rate_PostLockdown'] = delta['diversion_rate_post']-delta['diversion_rate_pre']

delta[['Change_Diversion_Rate_PostLockdown']].to_csv("../modeling_data/Y__Diversion_RateChange.csv")